In [41]:
import numpy as np 
import pandas as pd 

In [42]:
movies = pd.read_csv('data_set.csv')
copy = pd.read_csv('data_copy.csv')
movies = copy
movies = movies.dropna()
movies = movies.drop_duplicates()
#type
#title
#director
#cast -> top3
#country
#rating
#listed in
#description
#remove show_id , not top3 cast , date_added , release_year, duration from movies
movies = movies.drop(['date_added', 'release_year', 'duration'], axis=1)
# movies.isnull().sum()


In [43]:
def convert (obj):
    L = []
    c=0
    for i in obj:
        if c!=3:
            L.append(i)
            c+=1
        else:
            break
    return L


In [44]:
movies['cast'] = movies['cast'].apply(lambda x: convert(x.split(',')) if isinstance(x, str) else x)
movies['director'] = movies['director'].apply(lambda x: convert(x.split(',')) if isinstance(x, str) else x)
movies['country'] = movies['country'].apply(lambda x: convert(x.split(',')) if isinstance(x, str) else x)
movies['rating'] = movies['rating'].apply(lambda x: convert(x.split(',')) if isinstance(x, str) else x)
# movies['listed_in'] = movies['listed_in'].apply(lambda x: convert(x.split(',')) if isinstance(x, str) else x)
# movies['description'] = movies['description'].apply(lambda x: convert(x.split(',')) if isinstance(x, str) else x)
movies['cast'] = movies['cast'].apply(convert)
# movies['description']

In [45]:
# movies['cast'] = movies['cast'].apply(
#     lambda x: [name.strip() for name in x] if isinstance(x, list) else x
# )
# #convert cast to string
# movies['cast'] = movies['cast'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# movies.head(1)


In [46]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [47]:
movies['cast'] = movies['cast'].apply(collapse)
movies['director'] = movies['director'].apply(collapse)
movies['country'] = movies['country'].apply(collapse)
movies['rating'] = movies['rating'].apply(collapse)
# movies.head(1)

In [48]:
movies['description'] = movies['description'].apply(lambda x:x.split())
# movies

In [49]:
movies['tags'] = movies['type'] + ' ' + movies['description'].apply(' '.join) + ' ' + movies['cast'].apply(' '.join) + ' ' + movies['director'].apply(' '.join) + ' ' + movies['country'].apply(' '.join) + ' ' + movies['rating'].apply(' '.join) + ' ' + movies['listed_in']
new = movies.drop(['type', 'description', 'cast', 'director', 'country', 'rating', 'listed_in'], axis=1)
movies = new
# movies

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=6000,stop_words='english')
    

In [51]:
vector = cv.fit_transform(movies['tags']).toarray()
vector.shape

(5332, 6000)

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
similarity = cosine_similarity(vector)
# similarity[1]

In [54]:
#lowercae all the titles
movies['title'] = movies['title'].apply(lambda x:x.lower())


In [55]:
#resert the indes
movies = movies.reset_index(drop=True)


In [56]:
def recommend(movie):
    #lowercase the movie
    movie = movie.lower()
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [57]:
movie = 'zoom'
if movie in movies['title'].values:
    recommend(movie) 
else:
    print("Movie not found!")  


christmas with the kranks
the croods
beethoven's 2nd
monster house
liar, liar, vampire


In [58]:
import pickle

In [60]:
pickle.dump(movies,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))